In [4]:
import yaml
import pandas as pd
from bokeh.io import output_notebook
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure
from bokeh.transform import dodge
import math
from bokeh.layouts import gridplot
from bokeh.models.widgets import Panel, Tabs
from functools import reduce
output_notebook()

Loading BokehJS ...

In [15]:
def get_config():
    with open("config.yaml", 'r') as con:
        config = yaml.safe_load(con)
    return config

config = get_config()
taxa_counts = pd.read_csv(config["pooled_taxa"], sep="\t")
#taxa_counts = taxa_counts.set_index(["subject","Scientific Name"])
display(taxa_counts)

,Unnamed: 0,kingdom_id,phylum_id,class_id,order_id,family_id,genus_id,species_id,is_vegan
0,0,0,1224,1236,91347,543,561,562,False
1,1,0,976,200643,171549,0,909656,821,False
2,2,0,976,200643,171549,815,816,28116,False
3,3,0,1224,1236,91347,543,561,562,False
4,4,0,0,0,0,0,0,1977998,False
...,...,...,...,...,...,...,...,...,...
790466,387709,0,1239,909932,1843489,31977,39948,2582419,False
790467,387710,0,1224,1236,91347,543,561,562,False
790468,387711,0,0,0,0,0,0,77133,False
790469,387712,0,0,0,0,0,0,2049510,False


In [270]:
def pooled_df_means_std(taxa_counts):
    subjects = ["A", "B", "C", "D", "E"]
    df = taxa_counts.loc["A"].rename(columns={"Control":"Control A","Vegan" :"Vegan A"})
    for subject in subjects[1:]:    
        df1 = taxa_counts.loc[subject].rename(columns={"Control":f"Control {subject}","Vegan" :f"Vegan {subject}"})
        
        df = pd.merge(df, df1, on=["Scientific Name","taxa"])
    control = df[["taxa","Control A", "Control B", "Control C", "Control D", "Control E"]]
    control["Control mean"] = control[["taxa","Control A", "Control B", "Control C", "Control D", "Control E"]].mean(1)
    control["Control sem"] = control[["taxa","Control A", "Control B", "Control C", "Control D", "Control E"]].sem(1)
    vegan = df[["taxa","Vegan A", "Vegan B", "Vegan C", "Vegan D", "Vegan E"]]
    vegan["Vegan mean"] = vegan[["taxa","Vegan A", "Vegan B", "Vegan C", "Vegan D", "Vegan E"]].mean(1)
    vegan["Vegan sem"] = vegan[["taxa","Vegan A", "Vegan B", "Vegan C", "Vegan D", "Vegan E"]].sem(1)
    df_to_use = pd.merge(control,vegan, on=["Scientific Name", "taxa"])
    df_to_use = df_to_use[["taxa","Control mean", "Control sem", "Vegan mean", "Vegan sem"]]
    df_to_use = df_to_use.reset_index()
    return df_to_use.set_index(["taxa"])
pooled_df_means_std(taxa_counts)

/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,Scientific Name,Control mean,Control sem,Vegan mean,Vegan sem
taxa,,,,,
superkingdom,Bacteria,0.858794,0.024002,0.841200,0.030965
superkingdom,Unclassified,0.101542,0.017126,0.111495,0.026052
superkingdom,Eukaryota,0.027585,0.006982,0.035857,0.009777
superkingdom,Viruses,0.011914,0.001310,0.011361,0.001344
kingdom,Unclassified,0.972384,0.006986,0.964383,0.009761
...,...,...,...,...,...
species,Duncaniella dubosii,0.000260,0.000136,0.000301,0.000138
species,Haemophilus parainfluenzae,0.000244,0.000115,0.001131,0.000662
species,Muribaculum sp. TLL-A4,0.000341,0.000159,0.000347,0.000120


In [267]:
def blast_barplot(df,taxa):    
    mold = df.index.get_level_values("taxa")==taxa
    taxa_df = df[mold].set_index(["Scientific Name"])
    display(taxa_df)
    source = ColumnDataSource(data=taxa_df.iloc[0:4,])
    scientific_names = source.data["Scientific Name"]
    upper_value_control = source.data["Control mean"] + source.data["Control sem"]
    lower_value_control = source.data["Control mean"] - source.data["Control sem"] 
    upper_value_vegan =source.data["Vegan mean"] + source.data["Vegan sem"] 
    lower_value_vegan = source.data["Vegan mean"] - source.data["Vegan sem"] 
    
    
    
    p = figure(x_range=scientific_names, plot_width=700, plot_height=600)
    p.vbar(x=dodge('Scientific Name', -0.16, range=p.x_range), top="Vegan mean",color="#abdfff" ,source=source, width=0.3,line_color="#75cbff")
    p.vbar(x=dodge('Scientific Name', 0.16, range=p.x_range), top="Control mean",color="#ceffc4", source=source, width=0.3,line_color="#8dff75")
    
    p.segment([0.34,1.34,2.34,3.34], lower_value_control, [0.34,1.34,2.34,3.34], upper_value_control ,line_color="black")
    p.segment([0.66,1.66,2.66,3.66], lower_value_vegan ,[0.66,1.66,2.66,3.66], upper_value_vegan, line_color="black")

    p.rect([0.34,1.34,2.34,3.34], lower_value_control,0.01,line_color="black")
    p.rect([0.34,1.34,2.34,3.34], upper_value_control, 0.1,0.01,line_color="black")
    p.rect([0.66,1.66,2.66,3.66], upper_value_vegan, 0.1,0.01,line_color="black")
    p.rect([0.66,1.66,2.66,3.66], lower_value_vegan, 0.1,0.01,line_color="black")
    p.x_range.range_padding = 0.01
    p.xaxis.major_label_orientation = math.pi/6


    show(p)
blast_barplot(pooled_df_means_std(taxa_counts), "species")

/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/madelarambelje/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,Control mean,Control sem,Vegan mean,Vegan sem
Scientific Name,,,,
Escherichia coli,0.085341,0.021069,0.091895,0.021183
Bacteroides uniformis,0.064249,0.017932,0.068567,0.020312
uncultured bacterium,0.046064,0.008213,0.035562,0.005200
Prevotella copri,0.014886,0.010041,0.032672,0.030446
synthetic construct,0.048145,0.012959,0.058632,0.029646
...,...,...,...,...
Duncaniella dubosii,0.000260,0.000136,0.000301,0.000138
Haemophilus parainfluenzae,0.000244,0.000115,0.001131,0.000662
Muribaculum sp. TLL-A4,0.000341,0.000159,0.000347,0.000120
